# UCS757: Building Innovation Systems
## Assignment 1: Prediction in Time Series Dataset

### Sanyam Sharma
### 101903481
### 4CO19

### Loading and Cleaning Dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
road = pd.read_csv("road_dataset.csv")

In [3]:
road.rename(columns= {"Unnamed: 0":"Section"},inplace=True)

In [4]:
road.Section.fillna(method='ffill',inplace=True)
road.fillna(0,inplace=True)

In [6]:
test = road[road['year']==10]
test

,Section,year,Para-1,Para-2,Para-3,Para-4,Para-5,Para-6,Para-7,Para-8,Para-9,Para-10,Para-11,Para-12,Para-13
9,Sec_1,10,324.0,53.6,19.0,0.0,2,105,7567,1885,16.8,105.0,65.0,9,1.960
19,Sec_2,10,327.7,62.0,22.7,0.0,2,102,30525,10234,76.5,44.3,5.0,9,0.795
29,Sec_3,10,487.7,93.2,22.5,0.0,4,110,2140,1270,78.5,0.0,0.0,16,1.432
39,Sec_4,10,543.6,93.2,22.5,0.0,4,110,2140,1270,28.5,60.5,67.0,11,2.004
49,Sec_5,10,434.4,149.0,11.7,72.0,3,110,13142,6225,0.0,276.2,34.0,6,1.792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,Sec_97,10,514.0,1226.4,10.1,203.6,16,114,7672,303,219.7,206.2,10.0,8,1.607
979,Sec_98,10,901.0,1531.0,11.1,34.9,40,100,15150,1077,5.9,74.5,33.0,13,1.087
989,Sec_99,10,1181.0,1364.9,9.2,274.3,24,112,12500,880,131.4,72.4,18.0,12,0.996
999,Sec_100,10,662.0,1110.7,12.8,33.2,27,117,3324,1356,193.7,305.6,180.0,12,2.212


### Building Model

In [ ]:
!pip install catboost

In [96]:
import catboost as cb
from sklearn.metrics import accuracy_score

In [100]:
# Extract the training part of input dataset
move = 9
def extract_xtrain(df, row_start, col_start):
  row_end = row_start + move
  return df.iloc[row_start:row_end, 1:10]


# Extract the training part of output dataset
def extract_ytrain(df, row_start, col_start):
  row_end = row_start + move
  return df.iloc[row_start:row_end, col_start]


# Extract the testing part of input dataset
def extract_xtest(df, row_start):
  row_end = row_start + move
  return df.iloc[row_end, 1:10].values


# Extract the output part of ouput dataset
def extract_ytest(df, row_start, col_start):
  row_end = row_start + move
  return df.iloc[row_end, col_start]


# function to calcuate mean square error
def calcuate_lmse(predict_val, test_val):
  pred_diff = predict_val - test_val
  return (pred_diff)**2


# Get the XY Split
column_name = ["Para-"+str(i) for i in range(9,14)]
XY = []
def getXYsplit(df, row_start, row_end):
  XY = []
  XY.append(extract_xtrain(df, row_start, row_end))
  XY.append(extract_ytrain(df, row_start, row_end))
  XY.append(extract_xtest(df,row_start))
  XY.append(extract_ytest(df,row_start,row_end))
  return XY

In [113]:
from collections import defaultdict
from sklearn.metrics import mean_squared_error
# Create a dictionary for storing predicted and true values
pred_values = defaultdict(lambda : [])


target_col = [10,11,12,13,14]
root_mean_square_error = []
total_count = 0
curr_section = 0
total_squared_error = 0
max_val = 1000
for j in target_col:
  local_squared_error = 0
  l_count = 0
  LXY = [0,0,0,0]
  for i in range(0, max_val, 10):
    LXY = getXYsplit(road,i,j)
    x_train, y_train,x_test,y_test = LXY[0],LXY[1],LXY[2],LXY[3]
    # fit the model and predict

    train_dataset = cb.Pool(x_train, y_train) 
    test_dataset = cb.Pool(x_test.reshape(1,-1), y_test.reshape(1,-1))

    model = cb.CatBoostRegressor(loss_function='RMSE',logging_level = "Silent")

    grid = {'iterations': [100,150],
        'learning_rate': [0.03,0.1],
        'depth': [2,4,6],
        'l2_leaf_reg': [0.2,0.5]}
    
    try:
      model.grid_search(grid, train_dataset, verbose = False)
      ypred = model.predict(x_test.reshape(1,-1))[0]

      # store the predicted values
      pred_values[column_name[curr_section]].append(ypred)
      
      # store squared difference for RMSE

      # local_squared_error = calcuate_lmse(ypred, y_test) + local_squared_error
      total_squared_error = calcuate_lmse(ypred, y_test) + total_squared_error
      l_count += 1
      total_count += 1
    except:
      pred_values[column_name[curr_section]].append(0)



  # root_mean_square_error.append(local_squared_error)
  curr_section += 1
predicted_df = pd.DataFrame.from_dict(pred_values)

In [115]:
predicted_df

,Para-9,Para-10,Para-11,Para-12,Para-13
0,2.446259,54.642822,0.000000,7.616770,1.920570
1,52.619207,25.899800,0.916625,7.666520,0.792011
2,73.561719,0.000000,0.000000,13.873936,1.344596
3,16.404685,52.792113,52.810756,7.464705,1.785844
4,0.000000,255.548418,33.904269,5.959135,1.728446
...,...,...,...,...,...
95,0.000000,109.301095,14.733744,3.859975,1.397998
96,144.091177,184.172274,8.579520,7.222374,1.485208
97,4.377777,62.806572,22.907064,11.746483,1.068670
98,87.253238,57.457628,10.452608,10.363634,0.903523


### Result


In [114]:
# Final RMSE
total_rmse_by_n = total_squared_error/total_count
final_rmse = total_rmse_by_n**0.5

print("The RMSE is " + str(final_rmse))

The RMSE is 51.1006148431554
